In [40]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

# Sound recovery

In [41]:
def omp(A, y, s):
    # initialize
    residual = y
    index_set = []
    x = np.zeros(A.shape[1])

    for _ in range(s):
        # find the most correlated column
        idx = np.argmax(np.abs(A.T @ residual))
        index_set.append(idx)
        
        # update the solution
        A_selected = A[:, index_set]
        x_temp = np.linalg.lstsq(A_selected, y, rcond=None)[0]  # least square solution
        
        # update the residual
        residual = y - A_selected @ x_temp
        
        # if the residual is small enough, break
        if np.linalg.norm(residual) < 1e-3:
            break
    
    for i, idx in enumerate(index_set):
        x[idx] = x_temp[i]
    
    return x

In [42]:
import numpy as np
from scipy.linalg import lstsq
import sounddevice as sd
from PIL import Image
# The above defined libraries are sufficient to write the OMP code. Though you can use other libraries too. Do not use an OMP function from some library directly. 
# You have to code for it xD.

Image.MAX_IMAGE_PIXELS = None 

def loading_data(address):
    # Load compressed signal from .mat file
    compressedSignal = scipy.io.loadmat(f'{address}compressedSignal.mat')['compressedSignal']
    compressedSignal = compressedSignal.astype(np.float64)
    
    # Load compression matrix from .mat file
    compressionMatrix = scipy.io.loadmat(f'{address}compressionMatrix.mat')['compressionMatrix']
    
    # Load and process D_compressed from .tiff file
    D_compressed = Image.open(f'{address}CompressedBasis.tiff')
    D_compressed = np.array(D_compressed).astype(np.float64)
    D = D_compressed / 255.0 * 0.1284 - 0.0525
    
    return compressedSignal, D, compressionMatrix

In [43]:
compressedSignal, D, compressionMatrix = loading_data('./sound_data/')
print('compressedSignal:', compressedSignal.shape)
print('D:', D.shape)
print('compressionMatrix:', compressionMatrix.shape)

compressedSignal: (3000, 1)
D: (15980, 15980)
compressionMatrix: (3000, 15980)


In [44]:
import soundfile as sf
# save the compressed signal as .wav file
os.makedirs('./sound_results', exist_ok=True)
sf.write('./sound_results/compressedSignal.wav', compressedSignal, int(7350/2))

In [45]:
K_values = [10, 50, 100, 200, 300, 1000, 2000, 3000]

for K in K_values:
    selected_rows = compressionMatrix[:K, :]  # select the first K rows of the compression matrix
    compressed_signal_segment = compressedSignal[:K]  # select the first K elements of the compressed signal

    # omp
    sparsity = 100
    s = omp(selected_rows @ D, compressed_signal_segment, sparsity)

    # reconstruct the signal
    x = D @ s

    # save the reconstructed signal
    sf.write(f'./sound_results/reconstructed_sound_omp_K={K}.wav', x, 7350)


In [46]:
K_values = [10, 50, 100, 200, 300, 1000, 2000, 3000]

for K in K_values:
    selected_rows = compressionMatrix[:K, :]  # select the first K rows of the compression matrix
    compressed_signal_segment = compressedSignal[:K]  # select the first K elements of the compressed signal

    # least square
    s = np.linalg.lstsq(selected_rows @ D, compressed_signal_segment, rcond=None)[0]

    # reconstruct the signal
    x = D @ s

    # save the reconstructed signal
    sf.write(f'./sound_results/reconstructed_sound_lstsq_K={K}.wav', x, 7350)